In [12]:
# Imports
import sys
import json
from pathlib import Path
import numpy as np
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Ajouter le dossier racine
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

print(f"📁 Projet : {project_root}")
print(f"📂 Notebooks : {Path().absolute()}")

📁 Projet : c:\Users\rober\OneDrive\Bureau\PFE
📂 Notebooks : c:\Users\rober\OneDrive\Bureau\PFE\notebooks


In [13]:
"""
## 🔧 Fonction : Extraction et Normalisation des Skills de l'Offre
Optimisé avec skills_reference.json
"""

def normalize_skill(skill: str, variations_map: dict) -> str:
    """
    Normaliser un skill en utilisant le mapping de variations
    
    Args:
        skill: Compétence à normaliser
        variations_map: Dict {forme_canonique: [variations]}
        
    Returns:
        Forme canonique du skill
    """
    skill_clean = skill.lower().strip()
    skill_clean = skill_clean.replace('-', ' ').replace('_', ' ')
    skill_clean = ' '.join(skill_clean.split())
    
    # Chercher dans le mapping de variations
    for canonical, variations in variations_map.items():
        if skill_clean in variations or skill_clean == canonical:
            return canonical
    
    return skill_clean


def build_variations_map(skills_db: dict) -> dict:
    """
    Construire le mapping bidirectionnel canonical ↔ variations
    
    Args:
        skills_db: Contenu de skills_reference.json
        
    Returns:
        Dict {forme_canonique: [toutes_variations]}
    """
    variations_map = {}
    
    # Charger depuis la section "variations"
    if 'variations' in skills_db:
        for canonical, variations_list in skills_db['variations'].items():
            variations_map[canonical] = variations_list
    
    # Ajouter les skills techniques (eux-mêmes = forme canonique)
    if 'technical_skills' in skills_db:
        for skill in skills_db['technical_skills']:
            skill_lower = skill.lower().strip()
            if skill_lower not in variations_map:
                variations_map[skill_lower] = [skill_lower]
    
    return variations_map


def extract_job_skills_from_dict(job_dict: dict, skills_db: dict) -> List[str]:
    """
    Extraire et normaliser les skills d'une offre avec skills_reference.json
    
    Args:
        job_dict: Dictionnaire avec 'requirements' et 'nice_to_have'
        skills_db: Contenu de skills_reference.json
        
    Returns:
        Liste de skills normalisés et dédupliqués
    """
    import re
    
    # Construire le mapping de variations
    variations_map = build_variations_map(skills_db)
    
    # Créer une liste de tous les skills reconnus (formes canoniques + variations)
    all_known_skills = set()
    for canonical, variations in variations_map.items():
        all_known_skills.add(canonical)
        all_known_skills.update(variations)
    
    skills = []
    
    # 1. Requirements (priorité haute)
    if 'requirements' in job_dict and job_dict['requirements']:
        for req in job_dict['requirements']:
            # Extraire mots entre parenthèses
            match = re.search(r'\((.*?)\)', req)
            if match:
                # Ex: "Python (numpy, pandas)" → ["numpy", "pandas"]
                keywords = [k.strip() for k in match.group(1).split(',')]
                skills.extend(keywords)
            
            # Extraire skills connus du texte
            req_lower = req.lower()
            for known_skill in all_known_skills:
                # Pattern avec boundary pour éviter faux positifs
                pattern = r'\b' + re.escape(known_skill) + r'\b'
                if re.search(pattern, req_lower):
                    skills.append(known_skill)
    
    # 2. Nice-to-have (priorité moyenne)
    if 'nice_to_have' in job_dict and job_dict['nice_to_have']:
        for nice in job_dict['nice_to_have']:
            # Même processus
            match = re.search(r'\((.*?)\)', nice)
            if match:
                keywords = [k.strip() for k in match.group(1).split(',')]
                skills.extend(keywords)
            
            nice_lower = nice.lower()
            for known_skill in all_known_skills:
                pattern = r'\b' + re.escape(known_skill) + r'\b'
                if re.search(pattern, nice_lower):
                    skills.append(known_skill)
    
    # 3. Normaliser avec le mapping
    normalized = []
    seen = set()
    
    for skill in skills:
        if not skill or not skill.strip():
            continue
        
        skill_norm = normalize_skill(skill, variations_map)
        
        if skill_norm not in seen:
            normalized.append(skill_norm)
            seen.add(skill_norm)
    
    print(f"✅ {len(normalized)} skills uniques extraits de l'offre")
    if normalized:
        print(f"   Exemples : {', '.join(normalized[:5])}")
    
    return normalized


# Charger skills_reference.json
skills_db_path = project_root / "data" / "skills_reference.json"

if not skills_db_path.exists():
    print(f"❌ Fichier non trouvé : {skills_db_path}")
    skills_db = {}
else:
    with open(skills_db_path, 'r', encoding='utf-8') as f:
        skills_db = json.load(f)
    
    print("✅ skills_reference.json chargé")
    print(f"   • Technical skills : {len(skills_db.get('technical_skills', []))}")
    print(f"   • Variations mappées : {len(skills_db.get('variations', {}))}")
    print(f"   • Soft skills : {len(skills_db.get('soft_skills', []))}")

print("\n✅ Fonctions d'extraction et normalisation définies")

✅ skills_reference.json chargé
   • Technical skills : 171
   • Variations mappées : 45
   • Soft skills : 39

✅ Fonctions d'extraction et normalisation définies


In [14]:
"""
## 🤖 Chargement du modèle de vectorisation
"""

print("\n🔍 Chargement du modèle all-mpnet-base-v2...")
print("⏳ Première utilisation : téléchargement ~420 MB (1-2 min)")

try:
    model = SentenceTransformer('all-mpnet-base-v2')
    print("✅ Modèle chargé avec succès")
    print(f"   • Nom : all-mpnet-base-v2")
    print(f"   • Dimension des vecteurs : 768")
    print(f"   • Langue : Anglais (optimisé)")
    print(f"   • Performance : State-of-the-art pour matching sémantique")
except Exception as e:
    print(f"❌ Erreur : {e}")


🔍 Chargement du modèle all-mpnet-base-v2...
⏳ Première utilisation : téléchargement ~420 MB (1-2 min)
✅ Modèle chargé avec succès
   • Nom : all-mpnet-base-v2
   • Dimension des vecteurs : 768
   • Langue : Anglais (optimisé)
   • Performance : State-of-the-art pour matching sémantique


In [15]:
"""
## 📄 Chargement des compétences du CV
"""

# Charger les compétences extraites précédemment
cv_skills_path = project_root / "outputs" / "extracted_skills_simple.json"

if not cv_skills_path.exists():
    print(f"❌ Fichier non trouvé : {cv_skills_path}")
    print("⚠️  Exécutez d'abord le notebook 02_skills_extraction_simple.ipynb")
    cv_skills = []
else:
    with open(cv_skills_path, 'r', encoding='utf-8') as f:
        cv_data = json.load(f)
    
    cv_skills = cv_data.get('technical_skills', [])
    
    print(f"✅ Compétences CV chargées : {len(cv_skills)} skills")
    print(f"\n📋 Exemples (10 premières) :")
    for skill in cv_skills[:10]:
        print(f"  • {skill}")
    
    # Normaliser les skills du CV avec le même système
    variations_map = build_variations_map(skills_db)
    cv_skills_normalized = []
    seen = set()
    
    for skill in cv_skills:
        skill_norm = normalize_skill(skill, variations_map)
        if skill_norm not in seen:
            cv_skills_normalized.append(skill_norm)
            seen.add(skill_norm)
    
    print(f"\n✅ Skills CV normalisés : {len(cv_skills_normalized)} uniques")
    print(f"   Exemples : {', '.join(cv_skills_normalized[:5])}")

✅ Compétences CV chargées : 24 skills

📋 Exemples (10 premières) :
  • .net
  • artificial intelligence
  • big data
  • c
  • c#
  • c++
  • data analysis
  • data science
  • deep learning
  • docker

✅ Skills CV normalisés : 24 uniques
   Exemples : dotnet, artificialintelligence, bigdata, c, csharp


In [33]:
"""
## 💼 Création d'une offre d'emploi de test
"""

job_offer = {
    "job_id": "job_001",
    "category": "ml_engineer",
    "title": "Junior ML Engineer",
    "company": "AI Startup Paris",
    "location": "Paris, France",
    "type": "CDI",
    "experience": "0-2 ans",
    "salary": "35-45K€",
    "description": """Nous recherchons un Junior ML Engineer passionné pour rejoindre notre équipe R&D.

Responsabilités :
- Développer des modèles de Machine Learning (classification, régression)
- Entraîner et optimiser des réseaux de neurones avec PyTorch
- Déployer des modèles en production avec Docker
- Participer aux revues de code et à l'amélioration continue

Stack technique :
- Python, PyTorch, scikit-learn
- Docker, Git
- FastAPI pour les APIs
- PostgreSQL""",
    "requirements": [
        "Python (numpy, pandas, scikit-learn)",
        "Machine Learning basics (supervised learning)",
        "Git et GitHub",
        "Docker (notions de base)",
        "Anglais technique (lecture documentation)"
    ],
    "nice_to_have": [
        "PyTorch ou TensorFlow",
        "FastAPI ou Flask",
        "MLflow",
        "AWS ou GCP"
    ],
    "posted_date": "2026-01-08",
    "url": "https://example-jobs.com/1",
    "remote_ok": False,
    "applicants": 99
}

# Extraire les skills de l'offre
job_skills = extract_job_skills_from_dict(job_offer, skills_db)

print("\n" + "="*60)
print("💼 OFFRE D'EMPLOI")
print("="*60)
print(f"Titre : {job_offer['title']}")
print(f"Entreprise : {job_offer['company']}")
print(f"Localisation : {job_offer['location']}")
print(f"Expérience : {job_offer['experience']}")
print(f"Salaire : {job_offer['salary']}")

print(f"\n🔧 Compétences requises ({len(job_skills)}) :")
for i, skill in enumerate(job_skills, 1):
    print(f"  {i:2d}. {skill}")

✅ 17 skills uniques extraits de l'offre
   Exemples : numpy, pandas, scikitlearn, python, supervised learning

💼 OFFRE D'EMPLOI
Titre : Junior ML Engineer
Entreprise : AI Startup Paris
Localisation : Paris, France
Expérience : 0-2 ans
Salaire : 35-45K€

🔧 Compétences requises (17) :
   1. numpy
   2. pandas
   3. scikitlearn
   4. python
   5. supervised learning
   6. machinelearning
   7. github
   8. git
   9. notions de base
  10. docker
  11. lecture documentation
  12. pytorch
  13. tensorflow
  14. fastapi
  15. flask
  16. amazonwebservices
  17. googlecloud


In [34]:
"""
## 🎯 Fonction de Matching : APPROCHE 4 (Job → CV) - OPTIMISÉE
"""

def calculate_cv_job_match_approach4(
    cv_skills: List[str], 
    job_skills: List[str], 
    model
) -> Dict:
    """
    APPROCHE 4 : Skills Offre → CV (VERSION OPTIMISÉE)
    
    Pour chaque skill REQUIS par l'offre, trouver le meilleur match dans le CV.
    Optimisation : Encoder tous les skills UNE SEULE FOIS (cache).
    
    Args:
        cv_skills: Liste de compétences du CV (normalisées)
        job_skills: Liste de compétences requises par l'offre (normalisées)
        model: Modèle Sentence-Transformer
        
    Returns:
        Dict avec score, coverage, quality et détails
    """
    if not cv_skills or not job_skills:
        return {
            'overall_score': 0,
            'coverage': 0,
            'quality': 0,
            'covered_count': 0,
            'total_required': 0,
            'matches': []
        }
    
    print(f"\n🔍 APPROCHE 4 : Matching {len(job_skills)} skills offre ↔ {len(cv_skills)} skills CV")
    print("-"*60)
    
    # ✅ OPTIMISATION : Encoder tous les skills UNE SEULE FOIS
    print("⚡ Encodage des skills (cache)...")
    
    cv_embeddings = {
        skill: model.encode([skill.lower()], show_progress_bar=False)[0]
        for skill in cv_skills
    }
    
    job_embeddings = {
        skill: model.encode([skill.lower()], show_progress_bar=False)[0]
        for skill in job_skills
    }
    
    print(f"✅ {len(cv_embeddings)} skills CV encodés")
    print(f"✅ {len(job_embeddings)} skills offre encodés")
    print("-"*60)
    
    matches = []
    
    # Pour chaque skill de l'OFFRE
    for job_skill in job_skills:
        job_emb = job_embeddings[job_skill]
        
        best_similarity = 0
        best_cv_skill = None
        
        # Comparer avec CHAQUE skill du CV (réutilise le cache)
        for cv_skill in cv_skills:
            cv_emb = cv_embeddings[cv_skill]
            
            # Calcul de similarité avec embeddings pré-calculés
            similarity = cosine_similarity([job_emb], [cv_emb])[0][0] * 100
            
            if similarity > best_similarity:
                best_similarity = similarity
                best_cv_skill = cv_skill
        
        # Déterminer le statut
        if best_similarity >= 40:
            status = 'covered'
            match_level = 'high'
            emoji = "🟢"
        elif best_similarity >= 30:
            status = 'partial'
            match_level = 'medium'
            emoji = "🟡"
        else:
            status = 'missing'
            match_level = 'low'
            emoji = "🔴"
        
        matches.append({
            'job_skill': job_skill,
            'cv_skill': best_cv_skill,
            'similarity': best_similarity,
            'status': status,
            'match': match_level
        })
        
        # Afficher en temps réel
        print(f"{emoji} {job_skill:30s} → {best_cv_skill or 'N/A':30s} ({best_similarity:.1f}%)")
    
    # Trier par similarité décroissante
    matches = sorted(matches, key=lambda x: x['similarity'], reverse=True)
    
    # Calculer métriques
    covered = [m for m in matches if m['status'] == 'covered']
    
    # Coverage : % de skills de l'offre couverts
    coverage = (len(covered) / len(job_skills)) * 100 if job_skills else 0
    
    # Quality : Qualité moyenne des matchs couverts
    quality = sum(m['similarity'] for m in covered) / len(covered) if covered else 0
    
    # Score final : Moyenne pondérée
    overall_score = (coverage * 0.5) + (quality * 0.5)
    
    print("\n" + "="*60)
    print("📊 MÉTRIQUES")
    print("="*60)
    print(f"Coverage : {coverage:.1f}% ({len(covered)}/{len(job_skills)} skills couverts)")
    print(f"Quality  : {quality:.1f}% (qualité moyenne des matchs)")
    print(f"Score    : {overall_score:.1f}%")
    
    return {
        'overall_score': overall_score,
        'coverage': coverage,
        'quality': quality,
        'covered_count': len(covered),
        'total_required': len(job_skills),
        'matches': matches
    }


print("✅ Fonction de matching Approche 4 OPTIMISÉE définie")

✅ Fonction de matching Approche 4 OPTIMISÉE définie


In [35]:
"""
## 🔥 Calcul du matching CV ↔ Offre
"""

if cv_skills_normalized and job_skills:
    result = calculate_cv_job_match_approach4(cv_skills_normalized, job_skills, model)
    
    print("\n✅ Matching terminé avec succès")
    
    # Verdict
    if result['overall_score'] >= 70:
        verdict = "✅ EXCELLENT MATCH - Candidat très qualifié"
    elif result['overall_score'] >= 50:
        verdict = "🟡 BON MATCH - Candidat qualifié"
    else:
        verdict = "🔴 MATCH FAIBLE - Profil différent"
    
    print(f"\n{verdict}")
else:
    print("❌ Impossible de calculer le matching (pas de skills)")


🔍 APPROCHE 4 : Matching 17 skills offre ↔ 24 skills CV
------------------------------------------------------------
⚡ Encodage des skills (cache)...
✅ 24 skills CV encodés
✅ 17 skills offre encodés
------------------------------------------------------------
🟢 numpy                          → numpy                          (100.0%)
🟢 pandas                         → pandas                         (100.0%)
🟢 scikitlearn                    → scikitlearn                    (100.0%)
🟢 python                         → python                         (100.0%)
🟢 supervised learning            → machinelearning                (73.3%)
🟢 machinelearning                → machinelearning                (100.0%)
🟢 github                         → git                            (78.1%)
🟢 git                            → git                            (100.0%)
🔴 notions de base                → datascience                    (14.6%)
🟢 docker                         → docker                         (1

In [36]:
"""
## 📋 Résultats détaillés du matching
"""

if 'result' in locals() and result['matches']:
    print("\n" + "="*70)
    print("🎯 RÉSULTATS DÉTAILLÉS DU MATCHING")
    print("="*70)
    
    print(f"\n📊 SCORE GLOBAL : {result['overall_score']:.1f}%")
    print(f"   • Coverage : {result['coverage']:.1f}% ({result['covered_count']}/{result['total_required']} skills)")
    print(f"   • Quality  : {result['quality']:.1f}%")
    
    # Top matches
    print(f"\n🟢 COMPÉTENCES COUVERTES ({result['covered_count']}) :")
    print("-"*70)
    covered_matches = [m for m in result['matches'] if m['status'] == 'covered']
    for i, match in enumerate(covered_matches, 1):
        print(f"{i:2d}. {match['job_skill']:25s} → {match['cv_skill']:25s} ({match['similarity']:.1f}%)")
    
    # Skills manquants
    missing_matches = [m for m in result['matches'] if m['status'] == 'missing']
    if missing_matches:
        print(f"\n🔴 COMPÉTENCES MANQUANTES ({len(missing_matches)}) :")
        print("-"*70)
        for i, match in enumerate(missing_matches, 1):
            best = f"(meilleur match : {match['cv_skill']} {match['similarity']:.1f}%)" if match['cv_skill'] else ""
            print(f"{i:2d}. {match['job_skill']:30s} {best}")
    
    # Skills partiels
    partial_matches = [m for m in result['matches'] if m['status'] == 'partial']
    if partial_matches:
        print(f"\n🟡 COMPÉTENCES PARTIELLES ({len(partial_matches)}) :")
        print("-"*70)
        for i, match in enumerate(partial_matches, 1):
            print(f"{i:2d}. {match['job_skill']:25s} → {match['cv_skill']:25s} ({match['similarity']:.1f}%)")


🎯 RÉSULTATS DÉTAILLÉS DU MATCHING

📊 SCORE GLOBAL : 78.3%
   • Coverage : 76.5% (13/17 skills)
   • Quality  : 80.1%

🟢 COMPÉTENCES COUVERTES (13) :
----------------------------------------------------------------------
 1. numpy                     → numpy                     (100.0%)
 2. pandas                    → pandas                    (100.0%)
 3. python                    → python                    (100.0%)
 4. git                       → git                       (100.0%)
 5. machinelearning           → machinelearning           (100.0%)
 6. docker                    → docker                    (100.0%)
 7. scikitlearn               → scikitlearn               (100.0%)
 8. github                    → git                       (78.1%)
 9. supervised learning       → machinelearning           (73.3%)
10. tensorflow                → deeplearning              (53.2%)
11. pytorch                   → jupyter                   (50.9%)
12. flask                     → jupyter       

In [37]:
"""
## ⚖️ Comparaison : Approche 2 (CV → Job) vs Approche 4 (Job → CV)
"""

print("\n" + "="*70)
print("⚖️  COMPARAISON DES APPROCHES")
print("="*70)

# APPROCHE 2 : CV Skills → Job Description (ancien)
def calculate_approach2(cv_skills_norm, job_description, model):
    """Ancienne méthode : moyenne de tous les skills du CV"""
    job_sentences = [s.strip() for s in job_description.split('\n') if s.strip() and len(s.strip()) > 10]
    
    if not job_sentences:
        return 0
    
    job_embeddings = model.encode(job_sentences, show_progress_bar=False)
    
    scores = []
    for skill in cv_skills_norm:
        skill_embedding = model.encode([skill.lower()], show_progress_bar=False)
        similarities = cosine_similarity(skill_embedding, job_embeddings)[0]
        max_sim = max(similarities) * 100
        scores.append(max_sim)
    
    return sum(scores) / len(scores) if scores else 0

# Calculer avec Approche 2
score_approach2 = calculate_approach2(cv_skills_normalized, job_offer['description'], model)

print(f"\n📊 RÉSULTATS :")
print("-"*70)
print(f"{'Méthode':<45} {'Score':<10} {'Métrique'}")
print("-"*70)
print(f"{'APPROCHE 2 (CV → Job - Ancienne)':<45} {score_approach2:>6.1f}%   Moyenne tous skills CV")
print(f"{'APPROCHE 4 (Job → CV - Nouvelle)':<45} {result['overall_score']:>6.1f}%   Coverage + Quality")
print(f"  ↳ Coverage : {result['coverage']:.1f}%")
print(f"  ↳ Quality  : {result['quality']:.1f}%")
print("-"*70)

# Analyse
diff = result['overall_score'] - score_approach2
print(f"\n💡 ANALYSE :")
print(f"   Différence : {diff:+.1f} points")

if abs(diff) < 5:
    print("   ✅ Scores cohérents : Le CV est bien ciblé pour cette offre")
elif diff > 0:
    print("   ✅ Approche 4 plus juste : Focus sur les compétences requises")
else:
    print("   ⚠️  Approche 2 plus généreuse : Valorise la diversité des compétences")

print(f"\n📈 RECOMMANDATION :")
print("   L'Approche 4 est plus équitable car elle se concentre sur les")
print("   compétences REQUISES par l'offre, sans pénaliser les CVs avec")
print("   des compétences diversifiées.")


⚖️  COMPARAISON DES APPROCHES

📊 RÉSULTATS :
----------------------------------------------------------------------
Méthode                                       Score      Métrique
----------------------------------------------------------------------
APPROCHE 2 (CV → Job - Ancienne)                38.1%   Moyenne tous skills CV
APPROCHE 4 (Job → CV - Nouvelle)                78.3%   Coverage + Quality
  ↳ Coverage : 76.5%
  ↳ Quality  : 80.1%
----------------------------------------------------------------------

💡 ANALYSE :
   Différence : +40.2 points
   ✅ Approche 4 plus juste : Focus sur les compétences requises

📈 RECOMMANDATION :
   L'Approche 4 est plus équitable car elle se concentre sur les
   compétences REQUISES par l'offre, sans pénaliser les CVs avec
   des compétences diversifiées.


In [21]:
"""
## 🧪 Test d'équité : 3 profils différents
"""

print("\n" + "="*70)
print("🧪 TEST D'ÉQUITÉ : SPÉCIALISTE vs GÉNÉRALISTE vs PARTIEL")
print("="*70)

# Candidat A : Spécialiste (5 skills ciblés)
cv_A = ["python", "pytorch", "scikitlearn", "docker", "git"]

# Candidat B : Généraliste (même 5 skills + 5 autres hors sujet)
cv_B = ["python", "pytorch", "scikitlearn", "docker", "git", 
        "excel", "powerpoint", "photoshop", "wordpress", "management"]

# Candidat C : Partiel (seulement 2 skills pertinents)
cv_C = ["python", "pandas", "excel", "powerpoint"]

print(f"\n👤 Candidat A (Spécialiste) : {len(cv_A)} skills")
print(f"   {', '.join(cv_A)}")

print(f"\n👤 Candidat B (Généraliste) : {len(cv_B)} skills")
print(f"   {', '.join(cv_B[:5])} + {len(cv_B)-5} autres hors sujet")

print(f"\n👤 Candidat C (Partiel) : {len(cv_C)} skills")
print(f"   {', '.join(cv_C)}")

# Calculer scores
print("\n🔍 Calcul des scores...")
result_A = calculate_cv_job_match_approach4(cv_A, job_skills, model)
result_B = calculate_cv_job_match_approach4(cv_B, job_skills, model)
result_C = calculate_cv_job_match_approach4(cv_C, job_skills, model)

print("\n" + "="*80)
print("📊 RÉSULTATS COMPARATIFS")
print("="*80)
print(f"{'Candidat':<20} {'Score':<12} {'Coverage':<12} {'Quality':<12} {'Verdict'}")
print("-"*80)

for name, res in [("A (Spécialiste)", result_A), ("B (Généraliste)", result_B), ("C (Partiel)", result_C)]:
    if res['overall_score'] >= 70:
        verdict = "✅ Excellent"
    elif res['overall_score'] >= 50:
        verdict = "🟡 Bon"
    else:
        verdict = "🔴 Insuffisant"
    
    print(f"{name:<20} {res['overall_score']:>6.1f}%      {res['coverage']:>6.1f}%      {res['quality']:>6.1f}%      {verdict}")

print("-"*80)

print(f"\n💡 CONCLUSION :")
print(f"   • A et B ont des scores similaires malgré 5 vs 10 skills ✅")
print(f"   • C a un score plus faible car couvre moins de requirements ✅")
print(f"   • L'Approche 4 ne pénalise PAS la diversité des compétences ! 🎯")


🧪 TEST D'ÉQUITÉ : SPÉCIALISTE vs GÉNÉRALISTE vs PARTIEL

👤 Candidat A (Spécialiste) : 5 skills
   python, pytorch, scikitlearn, docker, git

👤 Candidat B (Généraliste) : 10 skills
   python, pytorch, scikitlearn, docker, git + 5 autres hors sujet

👤 Candidat C (Partiel) : 4 skills
   python, pandas, excel, powerpoint

🔍 Calcul des scores...

🔍 APPROCHE 4 : Matching 17 skills offre ↔ 5 skills CV
------------------------------------------------------------
⚡ Encodage des skills (cache)...
✅ 5 skills CV encodés
✅ 17 skills offre encodés
------------------------------------------------------------
🟢 numpy                          → python                         (56.8%)
🟢 pandas                         → python                         (50.6%)
🟢 scikitlearn                    → scikitlearn                    (100.0%)
🟢 python                         → python                         (100.0%)
🟢 supervised learning            → scikitlearn                    (47.1%)
🟢 machinelearning         